# Correction et Nettoyage des Anomalies dans les Données Étudiantes UCAD (2001-2024)

---


Ce notebook automatise la **correction des anomalies** dans les fichiers étudiants UCAD :

- Vérification de l’homogénéité des colonnes sur toutes les années
- Suppression des colonnes inutiles (plus de 80 % de valeurs manquantes)
- Harmonisation des noms de colonnes et des systèmes LMD/Classique
- Nettoyage général des valeurs manquantes



In [12]:
import pandas as pd 

Verifier si tous les fichiers ont les meme nons de colonnes 

In [13]:
#collone de reference
import pandas as pd
import glob  as glob
import os
data= pd.read_csv('C:/Users/HP/Desktop/mon_stage_senegal/data/Etudiants_2001_2024/Liste_globale_des_etudiants_tous_les_champs20232024.csv' , encoding='latin1' ,sep=';')
# Chemin vers le dossier contenant les fichiers
dossier = 'C:/Users/HP/Desktop/mon_stage_senegal/data/Etudiants_2001_2024'

colonnes_reference = set(data.columns)  # Colonnes de référence
fichiers_csv = glob.glob(os.path.join(dossier, '*.csv'))
#Vérifier les colonnes de chaque fichier
for fichier in fichiers_csv:
    data = pd.read_csv(fichier, encoding='latin1', sep=';')
    colonnes_actuelles = set(data.columns)
    if colonnes_actuelles != colonnes_reference:
        print(f"Les colonnes de {os.path.basename(fichier)} sont différentes de celles du fichier de référence(fichier 2001-2002).")
        print(f"Colonnes manquantes dans {os.path.basename(fichier)} :", colonnes_reference - colonnes_actuelles)
        print(f"Colonnes supplémentaires dans {os.path.basename(fichier)} :", colonnes_actuelles - colonnes_reference)
        
print(f"✅Les colonnes des fichiers sont identiques  celles du fichier de référence(fichier 2001-2002).")

✅Les colonnes des fichiers sont identiques  celles du fichier de référence(fichier 2001-2002).


In [14]:
#================concater les fichiers pour creer un dataframe complet 
if fichiers_csv:  # Vérifier qu'il y a des fichiers CSV dans le dossier
    df_total = pd.concat(
        (pd.read_csv(fichier, encoding='latin1', sep=';', low_memory=False)
         .assign(source=os.path.basename(fichier)) for fichier in fichiers_csv),
        ignore_index=True
    )
    print("Les fichiers ont été concaténés avec succès.")
else:
    print("Aucun fichier CSV trouvé dans le dossier spécifié.")
    



Les fichiers ont été concaténés avec succès.


In [15]:
df_total.columns

Index(['NUMERO', 'NUMERO_TABLE', 'INE', 'NUMERO_IDENTITIFICATION', 'NOM',
       'PRENOM', 'DATE_DE_NAISSANCE', 'LIEU_DE_NAISSANCE',
       'MAIL_INSTITUTIONNEL', 'SEXE', 'MENTION_BACC', 'ANNEE_BACC',
       'PAYS_DE_NAISSANCE', 'NATIONALITE', 'SERIE_BACC', 'REGION_DE_NAISSANCE',
       'ETABLISSMENT_CODE', 'ETABLISSEMENT', 'NIVEAU_SECTION',
       'SIGLE_NIVEAU_SECTION', 'COHORTE', 'ETAT_INSCRIPTION',
       'ANNEE_INSCRIPTION', 'ANNEE_UNIVERSITAIRE', 'TYPE_FORMATION',
       'CODE_NIVEAU', 'NIVEAU', 'DATE_INSCRIPTION', 'NIVEAU LMD',
       'NIVEAU LMD ET NON LMD', 'SYSTEME', 'DEPARTEMENT FORMATION', 'source'],
      dtype='object')

- harmoniser le nom des colonnes 

In [16]:
def hamoniser(data):
    #minuscule
    data.columns = data.columns.str.lower().str.replace(' ', '_')
    return data
    
df_total = hamoniser(df_total)   
    

In [17]:
df_total.columns

Index(['numero', 'numero_table', 'ine', 'numero_identitification', 'nom',
       'prenom', 'date_de_naissance', 'lieu_de_naissance',
       'mail_institutionnel', 'sexe', 'mention_bacc', 'annee_bacc',
       'pays_de_naissance', 'nationalite', 'serie_bacc', 'region_de_naissance',
       'etablissment_code', 'etablissement', 'niveau_section',
       'sigle_niveau_section', 'cohorte', 'etat_inscription',
       'annee_inscription', 'annee_universitaire', 'type_formation',
       'code_niveau', 'niveau', 'date_inscription', 'niveau_lmd',
       'niveau_lmd_et_non_lmd', 'systeme', 'departement_formation', 'source'],
      dtype='object')

- corriger l'anomalie lier au systeme LMD


In [ ]:
# importer le dataframe avec  les fomrations concernes 
data1 = pd.read_excel("C:/Users/HP/Downloads/NIVEAU FORMATION (1).xlsx")
def corriger_anomalie_lmd(df, df_niveaux):
    for _, row in df_niveaux.iterrows():
        niveau_formation = str(row['NIVEAU FORMATION'])
        niveau_code = str(row['NIVEAU'])
        mask = df['sigle_niveau_section'] == niveau_formation
        df.loc[mask, 'systeme'] = 'Classique'
        df.loc[mask, 'niveau_lmd_et_non_lmd'] = f"A{niveau_code}"
    return df

df_total = corriger_anomalie_lmd(df_total, data1)




c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [21]:
#verification voir si il reste des systeme lmd avant 2011
condition = df_total['annee_inscription'] < 2011 
df_lmd_avant_2011 = df_total[condition & (df_total['systeme'] == 'LMD')]
if not df_lmd_avant_2011.empty:
    print("Il y a des enregistrements avec le système LMD avant 2011 :")
    print("voici les formations concernées :")      
    print(df_lmd_avant_2011['sigle_niveau_section'].unique())
    


Il y a des enregistrements avec le système LMD avant 2011 :
voici les formations concernées :
['Quatrième Année du Deuxième Cycle des Etudes Médicales'
 "Maîtrise en Science de l'Information et de la Communication"
 "Deuxième Année Certif d'Aptit aux Fonct d'Insp de l'Education Populaire, de la Jeunesse et Sports"
 'Première Année du Deuxième Cycle des Etudes Pharmaceutiques'
 "Maîtrise Sciences et Techniques de l'Activité Physique et du Sport"
 "Thèse d'Etat Biologie végétale"
 'D.E.S.S. Certificat en Administration des Entreprises (I.F.A.C.E.)'
 "Thèse d'Etat Chimie et Biochimie des Produits Naturels"
 "Première Année Certif d'Aptit aux Fonct d'Inspec de l'Education Populaire, de la Jeunesse et Sports"
 'Licence 3 Chimie' "Thèse d'Etat Biologie Animale"
 'Thèse de Troisième Cycle Génie Informatique'
 'Licence 1 Mathématiques spécialité Enseignement (soir)'
 'Thèse de Troisième Cycle  Génie Mécanique'
 'Licence Professionnelle Science de Gestion'
 'Licence Informatique Professionnelle

> ⚠️ **Remarque importante :**
>
> Après correction, il reste encore des étudiants avec le système LMD avant 2011.  
> Cela signifie que le fichier des niveaux de formation utilisé pour corriger l’anomalie ne couvre pas toutes les formations concernées.  
> **Certaines formations n’ont donc pas été corrigées automatiquement.**
.

## **Correction des dates de naissance incorrectes**

**Objectif :**  
Harmoniser le format de la date de naissance, puis détecter les cas aberrants (par exemple, étudiants ayant obtenu le bac à un âge improbable : moins de 14 ans ou plus de 40 ans).  
Pour chaque étudiant, si plusieurs dates de naissance existent dans l'historique, choisir la date la plus logique 


In [ ]:
def corriger_anomalies_dates(df):
    # Harmoniser les dates de naissance
    df['annee_naissance'] = pd.to_datetime(df['date_de_naissance'], errors='coerce', format='%d/%m/%Y').dt.year
    df['annee_bacc'] = pd.to_numeric(df['annee_bacc'].astype(str).str.extract(r'(\d{4})')[0], errors='coerce')
    df['age_bac'] = df['annee_bacc'] - df['annee_naissance']

    # Sélection des étudiants avec âge aberrant
    anomalies = df[(df['age_bac'] < 14) | (df['age_bac'] > 40)]

    # Vérification de la cohérence des dates de naissance
    i = False
    for numero, group in anomalies.groupby('numero'):
        dates = group['date_de_naissance'].dropna().unique()
        if len(dates) != 1:
            i = True
            break

    if i:
        print("Il existe des personnes avec des anomalies d'age et des dates de naissance différentes dans l'historique.")
    else:
        print("Toutes les personnes ayant une anomalie d'age ont la même date de naissance dans toute la base.")

# Utilisation
corriger_anomalies_dates(df_total)




Toutes les personnes ayant une anomalie d'age ont la même date de naissance dans toute la base.


Conclusion : Apres analyse, toutes les personnes presentant une anomalie d'age (moins de 14 ans ou plus de 40 ans au moment du bac) ont la **même date de naissance enregistrée sur toutes leurs années d'inscription**.  
Il est quasiment impossible de savoir la date de naissance exacte de ces personnes 

## **Correction des seri de bac incorrectes**

**Objectif :**  
comme pour les date de naissance on utilise la meme logique 



In [ ]:
def detecter_anomalies_series(df):
    series_normales_avant_98 = [ "A1", "A2", "A3", "A4",  "C", "D", "E", "F1", "F2", "F3", "F4", "F5", "F6", "F7", "G1", "G2", "G3", "T1", "T2", "L'", "S'"  ]  
    series_normales_apres_98 = ["L1B", "S1A", "S1", "L1A", "S3", "S4", "T1", "STEG", "SEA", "F6","S5", "S2", "STIDD", "LA", "T2", "L-AR", "L2", "L'1"]
    series_ok = set(series_normales_avant_98 + series_normales_apres_98)

    df['serie_bacc'] = df['serie_bacc'].astype(str).str.strip().str.upper()

    # Détection des séries anormales
    anomalies = df[~df['serie_bacc'].isin(series_ok)]

    # Vérification de la cohérence pour chaque étudiant
    incoherent = False
    for numero, group in anomalies.groupby('numero'):
        series = group['serie_bacc'].dropna().unique()
        if len(series) != 1:
            incoherent = True 
            break

    if incoherent:
        print("Il existe des personnes avec une série de bac anormale et des séries différentes dans l'historique.")
    else:
        print("Toutes les personnes ayant une série de bac anormale ont la même série dans toute la base.")


detecter_anomalies_series(df_total) 


Toutes les personnes ayant une série de bac anormale ont la même série dans toute la base.


Conclusion : Apres analyse, toutes les personnes presentant une anomalie de seri de bac  ont la **même seri de bac enregistre lors de l'inscription**.  
Il est quasiment impossible de deviner les vrais seri de bac de ces personnes 

## **Correction des Mentions de bac incorrectes**

In [ ]:
mentions_normales  = ['AB', 'BI', 'TB', 'PA']

def detecter_anomalies_mentions(df):
    # Liste des mentions valides
    mentions_normales = ['AB', 'BI', 'TB', 'PA']

    # Harmonisation (majuscules, espaces retirés)
    df['mention_bacc'] = df['mention_bacc'].astype(str).str.strip().str.upper()

    # Détection des mentions anormales
    anomalies = df[~df['mention_bacc'].isin(mentions_normales)]

    # Vérification de la cohérence pour chaque étudiant
    incoherent = False
    for numero, group in anomalies.groupby('numero'):
        mentions = group['mention_bacc'].dropna().unique()
        if len(mentions) != 1:
            incoherent = True
            break 

    if incoherent:
        print("Il existe des personnes avec une mention de bac anormale et des mentions différentes dans l’historique.")
    else:
        print("Toutes les personnes ayant une mention de bac anormale ont la même mention dans toute la base.")

# Utilisation
detecter_anomalies_mentions(df_total)
      
 
 

Toutes les personnes ayant une mention de bac anormale ont la même mention dans toute la base.
